# Study Selection
partial version, contains full-text assessment 

In [29]:
import pandas as pd
import os
import json
import logging
import time

from utils.General.quicker import Quicker, QuickerData, QuickerStage
from utils.Evidence_Assessment.paper import Paper
from utils.Evidence_Assessment.outcome import Outcome
from utils.Evidence_Assessment.evidence import Evidence
from utils.logging import setup_logging

# setup_logging()

# Hyperparameters
YOUR_CONFIG_PATH = 'config/config.json' #  your config.json file
YOUR_DATASET_PATH = 'data/2021ACR RA' # your dataset path
YOUR_QUESTION_DECOMPOSITION_PATH =  'data/2021ACR RA/Question_Decomposition' # your question decomposition folder
YOUR_LITERATURE_SEARCH_PATH = 'data/2021ACR RA/Literature_Search' # your literature search folder
YOUR_STUDY_SELECTION_PATH = 'data/2021ACR RA/Study_Selection' # your study selection folder
YOUR_EVIDENCE_ASSESSMENT_PATH = 'data/2021ACR RA/Evidence_Assessment' # your evidence assessment folder
YOUR_PAPER_LIBRARY_PATH = 'data/2021ACR RA/Paper_Library' # your paper library folder


disease = 'Rheumatoid Arthritis (RA)' # Disease name or clinical topic of your clinical question. 
pico_idx = "ef0e4f95" # PICO index of the clinical question, saved in your PICO_Information.json file


study_design_list = ['randomized clinical trial'] # Study design list for inclusion criteria

In [30]:
# The following example code is used to load included records from the json file by Quicker.quicker_data.to_json method.
# record_included_path = "your_record_included_path" # json file. You can use the file saved by Quicker.quicker_data.to_json method.
# with open(record_included_path, 'r') as f:
#     record_included_dict = json.load(f)
# record_included_list = record_included_dict['record_included_studies'] 

# The included record is a dictionary satisfying the following format:
example_record = {"title": "Full dose, reduced dose or discontinuation of etanercept in rheumatoid arthritis.",
            "paper_uid": "566c1573",
            "pmid": "25873634",
            "reference": None,
            "authors": None,
            "year": "2015-04-14",
            "abstract": "BACKGROUND: The aim of the Dose Reduction or Discontinuation of Etanercept in Methotrexate-Treated Rheumatoid Arthritis Patients Who Have Achieved a Stable Low Disease Activity-State study was to investigate the effect of etanercept (ETN) dose maintenance, reduction or withdrawal on patients with rheumatoid arthritis (RA) who had already achieved stable low disease activity (LDA) on ETN 50 mg+methotrexate (MTX).\nMETHODS: Patients with RA (n=91) and stable LDA with ETN 50 mg once weekly (QW)+MTX were included. After 8 weeks with unchanged treatment, 73 patients were randomised in a double-blind design to ETN 50 mg QW+MTX (ETN50), ETN 25 mg QW+MTX (ETN25) or placebo QW+MTX (PBO) for 48 weeks. Patients who flared were declared failures and treated with open-label ETN50 until week 48. The primary outcome was the proportion of patients on ETN50 versus PBO who were non-failures after 48 weeks.\nRESULTS: The proportion of non-failure patients was significantly lower with ETN50 (52%; p=0.007) and ETN25 (44%; p=0.044) versus PBO (13%). Median time to failure was significantly shorter with PBO (6 weeks) compared with ETN50 (48 weeks; p=0.001) and ETN25 (36 weeks; p<0.001). The majority of patients who flared regained LDA with open-label ETN50 quickly. Adverse events were consistent with the known side effect profiles of these medications.\nCONCLUSIONS: In patients with established RA who have achieved stable LDA on ETN50+MTX, continuing both is superior to PBO+MTX. Reduced dose ETN was also more effective than PBO in maintaining a favourable response, suggesting that a maintenance strategy with reduced dose ETN may be possible in a number of patients with established RA.\nTRIAL REGISTRATION NUMBER: NCT00858780.",
            "url": "10.1136/annrheumdis-2014-205726",
            "doi": "10.1136/annrheumdis-2014-205726",
            "journal": None,
            "save_folder_path":'data/2021ACR RA/Paper_Library/PICOef0e4f95/566c1573' # optional, Path to save the paper library. If don't specify, you should implement the automatic function to match the paper save path.
            }
record_included_list = [example_record] # list of included records

In [31]:
quicker_data = QuickerData(disease=disease,pico_idx=pico_idx)
quicker = Quicker(
    config_path=YOUR_CONFIG_PATH,
    question_deconstruction_database_path=YOUR_QUESTION_DECOMPOSITION_PATH,
    literature_search_database_path=YOUR_LITERATURE_SEARCH_PATH,
    study_selection_database_path=YOUR_STUDY_SELECTION_PATH,
    evidence_assessment_database_path=YOUR_EVIDENCE_ASSESSMENT_PATH,
    quicker_data=quicker_data,
    paper_library_base=YOUR_PAPER_LIBRARY_PATH,
)

quicker.comparator_postfix_map = {} #! set {} if only one comparator


question_deconstruction_datapath = os.path.join(
    quicker.question_deconstruction_database_path, 'PICO_Information.json'
)
question_deconstruction_data = pd.read_json(
    question_deconstruction_datapath, dtype={'Index': str}
)
question_deconstruction_data = question_deconstruction_data[
    question_deconstruction_data['Index'] == pico_idx
]
original_qd_dict = question_deconstruction_data.to_dict(orient='records')

data_dict = dict(
    clinical_question=original_qd_dict[0]['Question'],
    population=original_qd_dict[0]['P'],
    intervention=original_qd_dict[0]['I'],
    comparison=original_qd_dict[0]['C'],
    study=study_design_list,
    outcome=original_qd_dict[0]['O'], #! optional, specify the outcomes you want to assess so that identified outcomes will be categorized into these outcomes

)

quicker._add_data_to_quickerdata_for_test(
    stage=QuickerStage.LITERATURE_SEARCH,
    default_value=data_dict,
)

quicker.set_inclusion_exclusion_criteria(inclusion_criteria='', exclusion_criteria='')

In [32]:
print(quicker)
# print(quicker.quicker_data)
print(quicker.quicker_data.check_stage_state())

Quicker: STUDY_SELECTION
StageState.NOT_STARTED


In [33]:
#! Please implement the get_pdf method of class Paper in utils\Evidence_Assessment\paper.py
_, _, _ = quicker.select_studies_by_full_text_assessment(record_included_list)